# Serving Models

This notebook contains a PyTorch model trained on the Iris dataset.  We will be using this model throughout the remainder of the model serving material.

In [8]:
import numpy as np
import pandas as pd

import mlflow
import mlflow.pyfunc

import warnings
warnings.filterwarnings('ignore')

import os, shutil
if not '_nbdir_' in globals(): 
  _nbdir_ = os.getcwd()
os.chdir(_nbdir_+'/..')

model_path = 'models/texrank'

# Model

In [2]:
from datasets import CNNDailyMail
import summa.summarizer as TextRank

from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.text_rank import TextRankSummarizer

example = CNNDailyMail(split='val', verbose=False)[0].src
model = lambda text: TextRank.summarize(text, words=50)

def predict(text, sentences=3):
  parser = PlaintextParser.from_string(text, Tokenizer("english"))
  summarizer = TextRankSummarizer()
  summary = summarizer(parser.document, sentences)
  summary = ' '.join(map(str, summary))
  return summary

print(predict(example))

for $ 89 , self-styled entrepreneur kyle waring will ship you 6 pounds of boston-area snow in an insulated styrofoam box -- enough for 10 to 15 snowballs , he says . so this week , waring began shipping larger amounts in the styrofoam cubes , which he promises will arrive anywhere in the u.s. in less than 20 hours . many of his customers appear to be companies in warm-weather states who are buying the snow as a gag , he said .


# MLflow PyFunc Packaging

Now that we have everything serialized to disk it's time to package everything up together.

In [3]:
# This will serve as an MLflow wrapper for the model
class ModelWrapper(mlflow.pyfunc.PythonModel):

  # Load in the model and all required artifacts
  # The context object is provided by the MLflow framework
  # It will contain all of the artifacts specified above
  def load_context(self, context):
    from models import TextRank
    self.model = TextRank()
    
  # Create a predict function for our models
  def predict(self, context, model_input):
    preds = []
    for row in model_input.iloc:
      preds.append(self.model(row['text'], row['mode'], int(row['length'])))
    return preds

In [4]:
# Inspect the default conda environment for MLflow
mlflow.pyfunc.get_default_conda_env()

{'name': 'mlflow-env',
 'channels': ['defaults'],
 'dependencies': ['python=3.8.1', {'pip': ['mlflow', 'cloudpickle==1.3.0']}]}

In [5]:
import sumy

# Let's create our own conda environment
conda_env = {
    'channels': ['defaults'],
    'dependencies': [
      'python=3.6.9',
      {
        'pip':[
          f'mlflow=={mlflow.__version__}',
          f'summa==1.2.0',
          f'sumy=={sumy.__version__}',
        ]
      }
    ],
    'name': 'mlflow-env-textrank'
}

In [6]:
# Package the model!
if os.path.isdir(model_path):  shutil.rmtree(model_path)

mlflow.pyfunc.save_model(path=model_path,
                         python_model=ModelWrapper(),
                         conda_env=conda_env,
                         artifacts={'example': 'assets/example.txt'},
                         code_path=['models.py'])

# Test Importing

In [9]:
import pandas as pd

# Load the model in `python_function` format
loaded_model = mlflow.pyfunc.load_model(model_path)

# Evaluate the model
with open(model_path+'/artifacts/example.txt', 'r') as f:
  example = f.read()
example = {'text': [example], 'mode': ['sentences'], 'length': [3]}
example = pd.DataFrame(example)
test_predictions = loaded_model.predict(example)

print(test_predictions)

["Health officials are also tracing the steps of care workers at the centre to ensure the virus hasn't spread to other parts of the community. She said the centre has done a good job letting family members know about the infections and steps it's taking to prevent the virus from spreading. Provincial efforts ramped up On Friday the province announced plans to ramp up its response to the coronavirus outbreak, outlining a wide-ranging provincial pandemic co-ordination plan to contain the spread of the COVID-19, the disease that results from the virus."]
